In [1]:
#Library importaion
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive


In [2]:
#mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#loading the dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sensor_infoFinal.csv')

In [4]:
data.tail()

RecordID           RecordDate                SOwner SensorName  \
1536     11677  2024-12-10 05:01:33  Tiwa, Oliver, Barbie        dht   
1537     11678  2024-12-10 05:01:34  Tiwa, Oliver, Barbie   humidity   
1538     11679  2024-12-10 05:01:37  Tiwa, Oliver, Barbie        ldr   
1539     11680  2024-12-10 05:01:38  Tiwa, Oliver, Barbie        dht   
1540     11681  2024-12-10 05:01:39  Tiwa, Oliver, Barbie   humidity   

      SensorReading  nodeID  
1536             25       2  
1537             76       2  
1538              0       2  
1539             -1       2  
1540             76       2

In [5]:
data.columns

Index(['RecordID', 'RecordDate', 'SOwner', 'SensorName', 'SensorReading',
       'nodeID'],
      dtype='object')

In [6]:
#Drop unwanted columns
data = data.drop(columns=['RecordID', 'SOwner'])

In [7]:

# Pivot the DataFrame
df_pivot = data.pivot_table(index=['RecordDate', 'nodeID'], columns='SensorName', values='SensorReading').reset_index()

# Flatten the MultiIndex columns
df_pivot.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in df_pivot.columns.values]

print("\nPivoted DataFrame:")
print(df_pivot)



Pivoted DataFrame:
               RecordDate  nodeID   dht  humidity  ldr
0     2024-12-10 03:38:11       2  -1.0       NaN -1.0
1     2024-12-10 03:38:12       2   NaN      61.0  NaN
2     2024-12-10 03:38:17       2   NaN       NaN  0.0
3     2024-12-10 03:38:19       2  29.0       NaN  NaN
4     2024-12-10 03:38:20       2   NaN      60.0  NaN
...                   ...     ...   ...       ...  ...
1408  2024-12-10 05:01:33       2  25.0       NaN  NaN
1409  2024-12-10 05:01:34       2   NaN      76.0  NaN
1410  2024-12-10 05:01:37       2   NaN       NaN  0.0
1411  2024-12-10 05:01:38       2  -1.0       NaN  NaN
1412  2024-12-10 05:01:39       2   NaN      76.0  NaN

[1413 rows x 5 columns]


In [8]:
df_pivot.columns

Index(['RecordDate', 'nodeID', 'dht', 'humidity', 'ldr'], dtype='object')

In [9]:
print(df_pivot.isnull().sum())

RecordDate      0
nodeID          0
dht           900
humidity      899
ldr           899
dtype: int64


In [11]:
from google.colab import files


# Save the DataFrame to a CSV file
df_pivot.to_csv('df_pivot.csv', index=False)

# Download the CSV file
files.download('df_pivot.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
print(df_pivot.isnull().sum())

RecordDate    0
nodeID        0
dht           0
humidity      0
ldr           0
dtype: int64


In [13]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RecordDate  1413 non-null   object 
 1   nodeID      1413 non-null   int64  
 2   dht         1413 non-null   float64
 3   humidity    1413 non-null   float64
 4   ldr         1413 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 55.3+ KB


In [14]:
# Convert to datetime with the correct format
df_pivot['RecordDate'] = pd.to_datetime(df_pivot['RecordDate'], format='%Y-%m-%d %H:%M:%S')

# Extract date and time components
df_pivot['date'] = df_pivot['RecordDate'].dt.date
df_pivot['time'] = df_pivot['RecordDate'].dt.time


In [15]:
# Rename the 'nodeID' column to 'Location'
df_pivot.rename(columns={'nodeID': 'Location'}, inplace=True)

In [16]:
df_pivot.head()

RecordDate  Location        dht   humidity          ldr  \
0 2024-12-10 03:38:11         2  -1.000000  68.800883    -1.000000   
1 2024-12-10 03:38:12         2  23.513693  61.000000  3985.088591   
2 2024-12-10 03:38:17         2  24.039262  60.880828     0.000000   
3 2024-12-10 03:38:19         2  29.000000  72.708914  4805.282327   
4 2024-12-10 03:38:20         2  24.692202  60.000000  3003.520835   

         date      time  
0  2024-12-10  03:38:11  
1  2024-12-10  03:38:12  
2  2024-12-10  03:38:17  
3  2024-12-10  03:38:19  
4  2024-12-10  03:38:20

In [17]:
# Convert the 'time' column to datetime format
df_pivot['time'] = pd.to_datetime(df_pivot['time'], format='%H:%M:%S', errors='coerce')

# Check for any missing or invalid values
if df_pivot['time'].isna().any():
    print("Warning: Some 'time' values could not be converted and were set to NaT")

# Extract hours, minutes, and seconds as numeric values
df_pivot['time_real'] = (
    df_pivot['time'].dt.hour.astype(float) +                 # Hours as float
    df_pivot['time'].dt.minute.astype(float) / 60 +          # Minutes as fraction of an hour
    df_pivot['time'].dt.second.astype(float) / 3600          # Seconds as fraction of an hour
)


In [18]:
df_pivot.head()

RecordDate  Location        dht   humidity          ldr  \
0 2024-12-10 03:38:11         2  -1.000000  68.800883    -1.000000   
1 2024-12-10 03:38:12         2  23.513693  61.000000  3985.088591   
2 2024-12-10 03:38:17         2  24.039262  60.880828     0.000000   
3 2024-12-10 03:38:19         2  29.000000  72.708914  4805.282327   
4 2024-12-10 03:38:20         2  24.692202  60.000000  3003.520835   

         date                time  time_real  
0  2024-12-10 1900-01-01 03:38:11   3.636389  
1  2024-12-10 1900-01-01 03:38:12   3.636667  
2  2024-12-10 1900-01-01 03:38:17   3.638056  
3  2024-12-10 1900-01-01 03:38:19   3.638611  
4  2024-12-10 1900-01-01 03:38:20   3.638889

In [19]:
df_pivot.drop(columns=['RecordDate', 'date', 'time' ], inplace=True)

In [20]:
# List of columns to convert to integers
columns_to_convert = ['humidity', 'ldr', 'time_real', 'dht']
# Convert specified columns to integers
df_pivot[columns_to_convert] = df_pivot[columns_to_convert].astype(int)

In [21]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Location   1413 non-null   int64
 1   dht        1413 non-null   int64
 2   humidity   1413 non-null   int64
 3   ldr        1413 non-null   int64
 4   time_real  1413 non-null   int64
dtypes: int64(5)
memory usage: 55.3 KB


In [22]:
#Predict temperature values given location, LDR value, humidity, time of day and any
#additional data point of your choice.

#Create test train sets
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


# Features and target variable
X = df_pivot.drop(['dht'], axis=1)
X= X.to_numpy()

y = df_pivot['dht']
y = y.to_numpy()



In [23]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Mean Absolute Error (MAE): {mae}")


Mean Absolute Error (MAE): 2.348452630224589


In [24]:
# Predict for the 50th entry
X_50th = X[50]
# Predict temperature for the 50th entry
predicted_temp = model.predict(X_50th.reshape(1, -1))
print(predicted_temp)


[23.8]
